In [ ]:
%reload_ext autoreload
%autoreload 2


%matplotlib inline

## Install FastAI

https://github.com/fastai/fastai

In short, `conda activate` the python environement, and run:  
```bash
conda install -c fastai -c pytorch -c anaconda fastai gh anaconda
```

Note: For more details about the modelling approach, please refer to:  
https://github.com/fastai/fastbook/blob/master/01_intro.ipynb

In [ ]:
from fastai.vision.all import *

In [ ]:
path = untar_data(URLs.CAMVID_TINY)

In [ ]:
def acc_camvid(inp, targ, void_code=0):
    targ = targ.squeeze(1)
    mask = targ != void_code
    return (inp.argmax(dim=1)[mask] == targ[mask]).float().mean()

In [ ]:
def get_y(o, path=path):
    return path / "labels" / f"{o.stem}_P{o.suffix}"

In [ ]:
dls = SegmentationDataLoaders.from_label_func(
    path,
    bs=8,
    fnames=get_image_files(path / "images"),
    label_func=get_y,
    codes=np.loadtxt(path / "codes.txt", dtype=str),
)
dls.one_batch()[0].shape[-2:], get_c(dls)

In [ ]:
learn = unet_learner(dls, resnet50, metrics=acc_camvid)
learn.fine_tune(20)

In [ ]:
learn.show_results(max_n=6, figsize=(7,8))

In [ ]:
learn.export("./fastai_unet.pkl")